In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. 대상 URL 및 헤더 설정
url = 'https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=07'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# 2. 페이지 요청
news_rss = requests.get(url, headers=headers)

if news_rss.status_code == 200:
    # RSS는 XML 구조이므로 원본 데이터를 넘겨줍니다.
    soup = BeautifulSoup(news_rss.content, 'html.parser')
    
    # 3. 'item' 태그(기사 덩어리)를 모두 선택
    items = soup.select("item")
    
    news_list = []
    
    for item in items:
        # 4. 제목(title)과 요약(description) 추출
        # .get_text()는 CDATA 태그 안의 알맹이만 쏙 빼줍니다.
        title = item.select_one("title").get_text(strip=True)
        description = item.select_one("description").get_text(strip=True)
        
        # 5. 리스트에 딕셔너리 형태로 저장
        news_list.append({
            'title': title,
            'description': description
        })
    
    # 6. 데이터프레임 생성
    df = pd.DataFrame(news_list)
    
    # 결과 확인 (상단 5개)
    print(f"✅ 총 {len(df)}개의 기사를 데이터프레임으로 만들었습니다.")
    display(df.head())

else:
    print(f"❌ 접속 실패: {news_rss.status_code}")

✅ 총 29개의 기사를 데이터프레임으로 만들었습니다.


,title,description
0,"<![CDATA[중국 드론 업체 DJI ""증거 없이 수입 금지""…미국 법원에 소송]]>",&#9650; 미국 상점에 전시된 DJI 무인기외국산 신형 무인기 및 관련 부품의 ...
1,"<![CDATA[미국, 인도·인도네시아 태양광 고율 관세…한화, 반사 이익?]]>",&#9650; 태양광 에너지미국 정부가 인도와 인도네시아 등지에서 생산되는 태양광 ...
2,<![CDATA[미국 최대 은행 JP모건도 AI발 대규모 인력 재배치 계획]]>,&#9650; 제이미 다이먼 JP모건 회장미국 최대 은행인 JP모건체이스의 제이미 ...
3,"<![CDATA[영국, 입국 전 전자여행허가 확대…한국 포함 85개국에 의무화]]>","&#9650; 영국의 한 공항영국이 25일부터 무비자 입국이 가능한 한국, 미국, ..."
4,<![CDATA['상호관세 돌려달라' 소송 봇물…로레알·다이슨도 합류]]>,&#9650; 미국 연방 대법원미국 연방 대법원이 최근 위법으로 판결한 도널드 트럼...


In [4]:
import html

# 클렌징 함수 (이전과 동일)
def clean_text(text):
    if not text: return ""
    cleaned = text.replace("<![CDATA[", "").replace("]]>", "")
    return html.unescape(cleaned).strip()

# [핵심] 이미 만들어진 df의 각 컬럼에 함수를 적용합니다.
df['title'] = df['title'].apply(clean_text)
df['description'] = df['description'].apply(clean_text)

display(df.head())

,title,description
0,"중국 드론 업체 DJI ""증거 없이 수입 금지""…미국 법원에 소송",▲ 미국 상점에 전시된 DJI 무인기외국산 신형 무인기 및 관련 부품의 미국 내 수...
1,"미국, 인도·인도네시아 태양광 고율 관세…한화, 반사 이익?",▲ 태양광 에너지미국 정부가 인도와 인도네시아 등지에서 생산되는 태양광 발전 기기에...
2,미국 최대 은행 JP모건도 AI발 대규모 인력 재배치 계획,▲ 제이미 다이먼 JP모건 회장미국 최대 은행인 JP모건체이스의 제이미 다이먼 최고...
3,"영국, 입국 전 전자여행허가 확대…한국 포함 85개국에 의무화","▲ 영국의 한 공항영국이 25일부터 무비자 입국이 가능한 한국, 미국, 캐나다, 호..."
4,'상호관세 돌려달라' 소송 봇물…로레알·다이슨도 합류,▲ 미국 연방 대법원미국 연방 대법원이 최근 위법으로 판결한 도널드 트럼프 대통령의...


In [5]:
df

,title,description
0,"중국 드론 업체 DJI ""증거 없이 수입 금지""…미국 법원에 소송",▲ 미국 상점에 전시된 DJI 무인기외국산 신형 무인기 및 관련 부품의 미국 내 수...
1,"미국, 인도·인도네시아 태양광 고율 관세…한화, 반사 이익?",▲ 태양광 에너지미국 정부가 인도와 인도네시아 등지에서 생산되는 태양광 발전 기기에...
2,미국 최대 은행 JP모건도 AI발 대규모 인력 재배치 계획,▲ 제이미 다이먼 JP모건 회장미국 최대 은행인 JP모건체이스의 제이미 다이먼 최고...
3,"영국, 입국 전 전자여행허가 확대…한국 포함 85개국에 의무화","▲ 영국의 한 공항영국이 25일부터 무비자 입국이 가능한 한국, 미국, 캐나다, 호..."
4,'상호관세 돌려달라' 소송 봇물…로레알·다이슨도 합류,▲ 미국 연방 대법원미국 연방 대법원이 최근 위법으로 판결한 도널드 트럼프 대통령의...
5,"멕시코 마약왕 사살 혼란에…혼다, 현지 공장 가동중단 뒤 재개",▲ 일본 자동차 브랜드 혼다멕시코 당국이 마약 카르텔 두목을 사살한 뒤 폭력 사태가...
6,"술 먹는 침팬지…""야생 소변 샘플 20개 검사, 17개서 알코올 대사물 검출""",▲ 나무 위에서 과일을 먹고 있는 우간다 응고고의 서부 침팬지아프리카 우간다에 서식...
7,"미국, 유럽·중동에 군용기 150대 이동…이라크전 이후 최대",▲ 제럴드 R. 포드함미국이 유럽과 중동 기지로 150대 넘는 군용기를 이동시킨 것...
8,"미국, 서안지구서 첫 영사 서비스…이스라엘 ""환영""",▲ 서안지구의 이스라엘 군용 차량미국이 요르단강 서안의 이스라엘 정착촌에서 처음으로...
9,"이란, 미국과 핵협상 앞두고 ""타결 가시권…외교 최우선해야""",▲ 아바스 아라그치 이란 외무장관이란은 미국을 상대로 3차 핵협상을 이틀 앞두고 '...
